# deep learning AI 만들기

## 단어 vector 사전 불러오기
## 학습, 테스트 영화 평가 데이터 불러오기
## 불러온 자연어 데이터를 벡터 숫자로 바꾸기
## tensor로 바꿔서 DataLoader에 싣기
## AI 만들기

## 검증

In [1]:
import sys
sys.path.append(r"C:\PyAI")

import custom
import numpy
import pickle #벡터 사전 불러오기
import pandas #자연어 데이터 불러오기
from IPython.display import display
from tqdm import tqdm

import torch # AI 만들기
import torch.nn as nn
from torch.utils.data import DataLoader

In [2]:
# 단어와 벡터 리스트 따로 되어있는 거 불러기기

with open("data/korean_word.pkl", mode = "rb") as f:
    word_dict = pickle.load(f)

with open("data/korean_vector_list.pkl", mode = "rb") as f:
    vector_list = pickle.load(f)

print(len(word_dict))
print(len(vector_list))
print(type(vector_list))

30187
30187
<class 'list'>


In [3]:
# 자연어 데이터 불러오기
train_df = pandas.read_csv("data/korean_movie_train.txt", sep = "\t", encoding = "UTF8")
test_df = pandas.read_csv("data/korean_movie_test.txt", sep = "\t", encoding = "UTF8")

display(train_df)
display(test_df)

,Unnamed: 0,document,label,word_len
0,0,아 더빙 . 진짜 짜증 나 네 목소리,0,8
1,1,흠 . 포스터 초딩 영화 줄 . 오버 연기 가볍 지 않 구나,1,13
2,2,너무재밓 었 다 그래서 보 는 것 을 추천 하 ㄴ다,0,11
3,3,교도소 이야기 이 구먼 . 솔직 히 재미 없 다 . 평점 조정,0,13
4,4,사이몬페그 익살 스럽 ㄴ 연기 가 돋보이 었 던 영화 ! 스파이더맨 늙 어 보이 기...,1,28
...,...,...,...,...
148958,149995,인간 이 문제 이 지 . 소 뭔 죄 이 ㄴ가 .,0,12
148959,149996,평점 이 너무 낮 어서 .,1,6
148960,149997,이것 이 뭐 ? 한국인 거들먹거리 고 필리핀 혼혈 착하 다 ?,0,12
148961,149998,청춘 영화 최 고봉 . 방황 우울 하 었 던 날 들 자화상,1,13


,Unnamed: 0,document,label,word_len
0,0,굳,1,1
1,2,뭐 이 야 이 평점 들 . 나쁘 지 않 지만 점 짜리 더더욱 아니 잖아,0,16
2,3,지루 하 지 않 은데 완전 막장 이 ㅁ . 돈 주 고 보 기 .,0,16
3,4,아니 었 어도 별 다섯 개 주 었 을 터 이 ㄴ데 . 왜 나오 어서 저 심기 를 불...,0,25
4,5,음악 이 주 되 ㄴ 최고 음악 영화,1,8
...,...,...,...,...
49617,49995,오랜만 평점 로 이 기 하 었 네 킹왕짱 쌈 뽕 하 ㄴ 영화 를 만나 었 습니다 강...,1,24
49618,49996,의지 박약 들 하 는 거 다 탈영 일단 주인공 김대희 닮 었 고 이등병 찐따,0,16
49619,49997,그림 좋 고 완성도 높 었 지만 . 보 는 내내 불안 하 게 만들 ㄴ다,0,16
49620,49998,절대 보 어서 안 되 ㄹ 영화 . 재미 없 고 기분 잡치 고 . 한 세트장 다 하 ...,0,22


In [4]:
#자연어 데이터를 숫자로 라벨링

str_list = train_df["document"].values.tolist()
word_labels = []
for s in str_list :
    temp = custom.word_vectorize(s, word_dict, 30)
    word_labels.append(temp)

target = train_df["label"].values.tolist()

print(len(word_labels))
print(len(target))
print(word_labels[1])


148963
148963
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4738, 307, 8232, 1, 309, 719, 307, 5485, 1667, 4479, 29, 50, 12978]


In [5]:
#train 데이터 싣기
device = "cuda" if torch.cuda.is_available() else "cpu"

tensor_x = torch.tensor(word_labels, dtype = torch.long, device = device)
tensor_t = torch.tensor(target, dtype = torch.long, device = device)

print(tensor_x.shape)
print(tensor_t.shape)

train_dataloader = DataLoader(list(zip(tensor_x, tensor_t)), batch_size = 300, shuffle=True)

torch.Size([148963, 30])
torch.Size([148963])


In [6]:
#자연어 데이터 숫자로 라벨링

str_list = test_df["document"].values.tolist()
word_labels = []
for s in str_list :
    temp = custom.word_vectorize(s, word_dict, 30)
    word_labels.append(temp)

target = test_df["label"].values.tolist()

print(len(word_labels))
print(len(target))

49622
49622


In [7]:
#test 데이터 싣기
tensor_x = torch.tensor(word_labels, dtype = torch.long, device = device)
tensor_t = torch.tensor(target, dtype = torch.long, device = device)

test_dataloader = DataLoader(list(zip(tensor_x, tensor_t)), batch_size = 1000, shuffle=False)

print(tensor_x.shape)
print(tensor_t.shape)

torch.Size([49622, 30])
torch.Size([49622])


In [8]:
#AI 만들기

# x,t 입력은 dataloader에서 처리

# 함수들 만들고
class NN(nn.Module) :
    def __init__(self, vector_tensor) :
        super().__init__()
        self.embedding = nn.Embedding.from_pretrained(vector_tensor, freeze = True, padding_idx = 0) #freeze : 해당 벡터사전 고정, padding_idx = 패딩 인스스
        self.rnn = nn.LSTM(200,20,batch_first = True)
        self.f = nn.Sequential(
            nn.Linear(20, 2)
        )
    def forward(self, x) :
        x = self.embedding(x)
        y, h = self.rnn(x)
        y = y[:,-1,:]
        y = self.f(y)
        return y
vector_tensor = torch.tensor(vector_list, dtype = torch.float)
F = NN(vector_tensor).to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 1)
epoch = 100
prev_acc = 0
cnt = 0

for e in range(epoch) :
    loss_sum = 0
    for x, t in train_dataloader :
# 순전파
        y = F(x)
# 손실함수
        loss = loss_function(y, t)
        loss_sum += loss.item()
# 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_sum /= len(train_dataloader)
    
# 중간 ACC, earlystopper 구현
    correct = 0
    total = 0
    for x, t in test_dataloader :
        y = F(x)
        correct += (y.argmax(dim = -1) == t).sum().item()
        total += len(x)
    acc = correct / total

    if acc <= prev_acc + 0.001 :
        cnt += 1
    else :
        cnt = 0
        prev_acc = acc
    print(f"epoch {e+1} | loss {loss_sum} | acc {acc} | cnt {cnt}")
    if cnt >= 5 :
        print("--train stopped--")
        break

epoch 1 | loss 0.5299001969322112 | acc 0.7775180363548426 | cnt 0
epoch 2 | loss 0.4499623060706156 | acc 0.7923703196162992 | cnt 0
epoch 3 | loss 0.42443865675561626 | acc 0.7974084075611624 | cnt 0
epoch 4 | loss 0.40857575141208274 | acc 0.804784168312442 | cnt 0
epoch 5 | loss 0.39747814135532267 | acc 0.8075450405062271 | cnt 0
epoch 6 | loss 0.3874569197056039 | acc 0.8112530732336464 | cnt 0
epoch 7 | loss 0.38070654971019124 | acc 0.8147595824432712 | cnt 0
epoch 8 | loss 0.3742483291227813 | acc 0.8014993349723912 | cnt 1
epoch 9 | loss 0.36814674297328925 | acc 0.8158881141429205 | cnt 0
epoch 10 | loss 0.36311738687741685 | acc 0.8142356212970053 | cnt 1
epoch 11 | loss 0.3579318585769991 | acc 0.821611382048285 | cnt 0
epoch 12 | loss 0.35462729156137235 | acc 0.820805287977107 | cnt 1
epoch 13 | loss 0.3509159434729179 | acc 0.8215106202893878 | cnt 2
epoch 14 | loss 0.34786080486577764 | acc 0.8222562573052276 | cnt 3
epoch 15 | loss 0.3440856300369355 | acc 0.818165329

In [9]:
#신경망 저장

torch.save(F.to("cpu"), "korean_movie_2.pt")